In [ ]:
#installs
!pip install nltk
!pip install scikit-learn
!pip install pandas
!pip install demoji
!pip install pyspellchecker

In [ ]:
#imports
import nltk
import sklearn
import pandas as pd
import os
import re


In [ ]:
#separates file names into comments vs submissions for each class

import os
import re

unpath = 'undiagnosed_user_data/'
dpath = 'diagnosed_user_data/'
unlist = os.listdir(unpath)
dlist = os.listdir(dpath)
dsubmissions = []
dcomments = []
usubmissions = []
ucomments = []

for file in dlist:
    if '_submission_data.txt' in file:
        dsubmissions.append(file)
    elif '_comment_data.txt' in file:
        dcomments.append(file)
    else:
        print(f'error with file: {file}')
        continue
        
for file2 in unlist:
    if '_submission_data.txt' in file2:
        usubmissions.append(file2)
    elif '_comment_data.txt' in file2:
        ucomments.append(file2)
    else:
        print(f'error with file: {file2}')
        continue

In [ ]:
#preprocessing methods
def submission_remove_lines(filename, name, isDiagnosed):
    tempfile = open(filename, 'r')
    front = ''
    if isDiagnosed:
        front += 'diagnosed_current/'
    else:
        front += 'undiagnosed_current/'
    outfilename = '' + front + name + '_submissions_stripped.txt'
    
    outfile = open(outfilename, 'w')
    for line in tempfile:
        line_list = line.split(':;')
        if len(line_list) != 15:
            continue
        if line_list[0] != name:
            continue
        
        replacement_string = line.replace('\n', ' ')
        outfile.write(replacement_string.strip())
        outfile.write('\n')
    tempfile.close()
    outfile.close()
        
    return outfilename

def comment_remove_lines(filename, name, isDiagnosed):
    tempfile = open(filename, 'r')
    front = ''
    if isDiagnosed:
        front += 'diagnosed_current/'
    else:
        front += 'undiagnosed_current/'
    outfilename = '' + front + name + '_comments_stripped.txt'
    outfile = open(outfilename, 'w')
    for line in tempfile:
        line_list = line.split(':;')
        if len(line_list) != 12:
            continue
        if line_list[0] != name:
            continue
        replacement_string = line.replace('\n', ' ')
        
        outfile.write(replacement_string.strip())
        outfile.write('\n')
        
    tempfile.close()
    outfile.close()
    return outfilename

#returns 4 lists of filenames with files where extra spaces are removed,
def preprocess_one(diagcoms, diagsubs, uncoms, unsubs):
    diagnosed_comment_files = [] #keeps a list of filenames for diagnosed users
    diagnosed_submission_files = [] #keeps a list of filenames for diagnosed users
    undiagnosed_comment_files = [] #keeps a list of filenames for undiagnosed users
    undiagnosed_submission_files = [] #keeps a list of filenames for undiagnosed users
    for diag in diagcoms:
        dfilename = 'diagnosed_user_data/' + diag
        diagname = diag.split('_comment')[0]
        currentfile = comment_remove_lines(dfilename, diagname, True)
        newfname = 'demoji_diagnosed/' + currentfile.split('diagnosed_current/')[1]
        currentfile = demoji_stuff(currentfile, newfname, True, False)
        diagnosed_comment_files.append(currentfile)
    
        
    for diag in diagsubs:
        dfilename = 'diagnosed_user_data/' + diag
        diagname = diag.split('_submission')[0]
        currentfile= submission_remove_lines(dfilename, diagname, True)
        newfname = 'demoji_diagnosed/' + currentfile.split('diagnosed_current/')[1]
        currentfile = demoji_stuff(currentfile, newfname, True, True)
        diagnosed_submission_files.append(currentfile)
        
        
    for un in uncoms:
        ufilename =  'undiagnosed_user_data/' + un
        unname = un.split('_comment')[0]
        currentfile = comment_remove_lines(ufilename, unname, False)
        newfname = 'demoji_undiagnosed/' + currentfile.split('undiagnosed_current/')[1]
        currentfile = demoji_stuff(currentfile, newfname, False, False)
        undiagnosed_comment_files.append(currentfile)
       
    for un in unsubs:
        ufilename =  'undiagnosed_user_data/' + un
        unname = un.split('_submission')[0]
        currentfile = submission_remove_lines(ufilename, unname, False)
        newfname = 'demoji_undiagnosed/' + currentfile.split('undiagnosed_current/')[1]
        currentfile = demoji_stuff(currentfile, newfname, False, True)
        undiagnosed_submission_files.append(currentfile)
        
    #at this point all extra new line characters should be removed and demojied
    
    return diagnosed_comment_files, diagnosed_submission_files, undiagnosed_comment_files, undiagnosed_submission_files

In [ ]:
#preprocessing imports
import os
import demoji
import re

#done for now unless i want to incorporate emojis for deeper textual analysis
#removes emojis from text posts, writes emoji counts to files: one for diagnosed, one for undiagnosed
def demoji_stuff(filename, newfilename, isDiagnosed, isSubmission):
    oldfile = open(filename, 'r')
    newfile = open(newfilename, 'w')
    emoji_count = 0
    for line in oldfile:
        emoji_count += len(demoji.findall_list(line))
        templine = demoji.replace(line, "")#replaces all emojis with empty space, may change this to description later
        newfile.write(templine.strip())
        newfile.write('\n')
    oldfile.close()
    newfile.close()
    if isDiagnosed:
        emojifile = open('emojis_diagnosed.txt', 'a')
        if isSubmission:
            name = newfilename.split('diagnosed/')[1].split('_submission')[0]
        else:
            name = newfilename.split('diagnosed/')[1].split('_comment')[0]
        emojifile.write(str(name))
        emojifile.write(':')
        emojifile.write(str(emoji_count))
        emojifile.write('\n')
        emojifile.close()
    else:
        if isSubmission:
            name = newfilename.split('undiagnosed/')[1].split('_submission')[0]
        else:
            name = newfilename.split('undiagnosed/')[1].split('_comment')[0]
        emojifile = open('emojis_undiagnosed.txt', 'a')
        emojifile.write(str(name))
        emojifile.write(':')
        emojifile.write(str(emoji_count))
        emojifile.write('\n')
        emojifile.close()
    return newfilename

def demoji2(dlist, ulist):
    for user in dlist:
        comm_filename = 'processed_diagnosed/' + user + '_comment_data.txt'
        sub_filename = 'processed_diagnosed/' + user + '_submission_data.txt'
        new_comm_filename = 'demoji_diagnosed/' + user +'_comments_stripped.txt'
        new_sub_filename = 'demoji_diagnosed/' + user + '_submissions_stripped.txt'
        
        comm_file = open(comm_filename, 'r')
        new_comm_file = open(new_comm_filename, 'w')
        
        for line in comm_file:
            templine = demoji.replace(line, "")#replaces all emojis with empty space, may change this to description later
            new_comm_file.write(templine.strip())
            new_comm_file.write('\n')
        
        comm_file.close()
        new_comm_file.close()
        
        sub_file = open(sub_filename, 'r')
        new_sub_file = open(new_sub_filename, 'w')
        
        for line in sub_file:
            templine = demoji.replace(line, "")#replaces all emojis with empty space, may change this to description later
            new_sub_file.write(templine.strip())
            new_sub_file.write('\n')
        
        sub_file.close()
        new_sub_file.close()
        
    for user in ulist:
        comm_filename = 'processed_undiagnosed/' + user + '_comment_data.txt'
        sub_filename = 'processed_undiagnosed/' + user + '_submission_data.txt'
        new_comm_filename = 'demoji_undiagnosed/' + user +'_comments_stripped.txt'
        new_sub_filename = 'demoji_undiagnosed/' + user + '_submissions_stripped.txt'
        comm_file = open(comm_filename, 'r')
        new_comm_file = open(new_comm_filename, 'w')
        
        for line in comm_file:
            templine = demoji.replace(line, "")#replaces all emojis with empty space, may change this to description later
            new_comm_file.write(templine.strip())
            new_comm_file.write('\n')
        
        comm_file.close()
        new_comm_file.close()
        
        sub_file = open(sub_filename, 'r')
        new_sub_file = open(new_sub_filename, 'w')
        
        for line in sub_file:
            templine = demoji.replace(line, "")#replaces all emojis with empty space, may change this to description later
            new_sub_file.write(templine.strip())
            new_sub_file.write('\n')
        
        sub_file.close()
        new_sub_file.close()

In [ ]:

#removing whitespace from files
def get_all_users():
    all_uusers = []
    all_dusers = []
    
    unanxietyfile = open('proven_all_diagnosed.txt', 'r')
    for user in unanxietyfile:
        all_dusers.append(user.strip())
           
    unanxietyfile2 = open('proven_all_undiagnosed.txt', 'r')
    for user in unanxietyfile2:
        all_uusers.append(user.strip())
        
    return all_dusers, all_uusers

def remove_lines_comment(dusers, uusers):
    for user in dusers:
        filename = 'diagnosed_user_data/' + user + '_comment_data.txt'
        fstring = open(filename).read().replace('\n', ' ')
        newname = 'processed_diagnosed/' + user + '_comment_data.txt'
    
        newfile = open(newname, 'w')
        flist = fstring.split(':;')
        
        i = 1
        for thing in flist:
            if i % 12 == 0:
                temp = thing.split()
               # print(temp)
                newfile.write(temp[0])
                newfile.write('\n')
                if len(temp) > 1:
                    newfile.write(temp[1])
                    newfile.write(':;')
                i+=1
            else:
                newfile.write(thing.rstrip())
                newfile.write(':;')
            i += 1
        
        newfile.close()
        
    for user in uusers:
        filename = 'undiagnosed_user_data/' + user + '_comment_data.txt'
        fstring = open(filename).read().replace('\n', ' ')
        newname = 'processed_undiagnosed/' + user + '_comment_data.txt'
    
        newfile = open(newname, 'w')
        flist = fstring.split(':;')
        
        i = 1
        for thing in flist:
            if i % 12 == 0:
                temp = thing.split()
                #print(temp)
                newfile.write(temp[0])
                newfile.write('\n')
                if len(temp) > 1:
                    newfile.write(temp[1])
                    newfile.write(':;')
                i+=1
            else:
                newfile.write(thing.rstrip())
                newfile.write(':;')
            i += 1
        
        newfile.close()
def remove_lines_submission(dusers, uusers):
    for user in dusers:
        filename = 'diagnosed_user_data/' + user + '_submission_data.txt'
        fstring = open(filename).read().replace('\n', ' ')
        newname = 'processed_diagnosed/' + user + '_submission_data.txt'
    
        newfile = open(newname, 'w')
        flist = fstring.split(':;')
        
        i = 1
        for thing in flist:
            if i % 15 == 0:
                temp = thing.split()
                #print(temp)
                
                
                if len(temp) > 1:
                    for j in range(len(temp)-1):
                        newfile.write(temp[j])
                        newfile.write(' ')
                    
                    
                    last_thing = temp[len(temp)-1]
                    if last_thing.strip() == user.strip():
                        newfile.write('\n')
                        newfile.write(temp[len(temp)-1])
                        newfile.write(':;')
                    else:
                        newfile.write(temp[len(temp)-1])
                        newfile.write('\n')
                    
                else:
                    newfile.write(temp[0])
                    newfile.write('\n')
              #  print(temp[len(temp)-1])
                i+=1
            else:
                newfile.write(thing.rstrip())
                newfile.write(':;')
            i += 1
        
        newfile.close()
        
    for user in uusers:
        filename = 'undiagnosed_user_data/' + user + '_submission_data.txt'
        fstring = open(filename).read().replace('\n', ' ')
        newname = 'processed_undiagnosed/' + user + '_submission_data.txt'
        
        newfile = open(newname, 'w')
        flist = fstring.split(':;')
        
        i = 1
        for thing in flist:
            if i % 15 == 0:
                temp = thing.split()
                #print(temp)
                
                
                if len(temp) > 1:
                    for j in range(len(temp)-1):
                        newfile.write(temp[j])
                        newfile.write(' ')
                    
                    
                    last_thing = temp[len(temp)-1]
                    if last_thing.strip() == user.strip():
                        newfile.write('\n')
                        newfile.write(temp[len(temp)-1])
                        newfile.write(':;')
                    else:
                        newfile.write(temp[len(temp)-1])
                        newfile.write('\n')
                    
                else:
                    newfile.write(temp[0])
                    newfile.write('\n')
               # print(temp[len(temp)-1])
                i+=1
            else:
                newfile.write(thing.rstrip())
                newfile.write(':;')
            i += 1
        
        
        newfile.close()



In [ ]:
dlist, ulist = get_all_users()
remove_lines_comment(dlist,ulist)
remove_lines_submission(dlist,ulist)

In [ ]:
dlist, ulist = get_all_users()
demoji2(dlist, ulist)

In [ ]:
#dsubmissions = []
#dcomments = []
#usubmissions = []
#ucomments = []
dcomments2, dsubmissions2, ucomments2, usubmissions2 = preprocess_one(dcomments, dsubmissions, ucomments, usubmissions)
     

In [ ]:
#fixes emoji files
unemojifile = open('emojis_undiagnosed.txt','r')
demojifile = open('emojis_diagnosed.txt', 'r')
demojidict = {}
unemojidict = {}

for line in unemojifile:
    temp = line.split(':')
    if len(temp) != 2:
        print(f'error with {line}')
        continue
    name = temp[0]
    count = int(temp[1])
    if name in unemojidict:
        unemojidict[name] += count
    else:
        unemojidict[name] = count
for line in demojifile:
    temp = line.split(':')
    if len(temp) != 2:
        print(f'error with {line}')
        continue
    name = temp[0]
    count = int(temp[1])
    if name in demojidict:
        demojidict[name] += count
    else:
        demojidict[name] = count
unemojifile.close()
demojifile.close()

unemojiout = open('emojis_undiagnosed_unique.txt', 'w')
demojiout = open('emojis_diagnosed_unique.txt', 'w')
for name in unemojidict:
    unemojiout.write(name)
    unemojiout.write(':')
    unemojiout.write(str(unemojidict[name]))
    unemojiout.write('\n')
for name in demojidict:
    demojiout.write(name)
    demojiout.write(':')
    demojiout.write(str(demojidict[name]))
    demojiout.write('\n')
unemojiout.close()
demojiout.close()

In [ ]:
#emoji distributions
def emoji_distributions():
   
    undiagnosed_emojis = []
    diagnosed_emojis = []

    unemojifile = open('emojis_undiagnosed_unique.txt','r')
    demojifile = open('emojis_diagnosed_unique.txt', 'r')

    for line in unemojifile:
        temp = line.split(':')
        if len(temp) != 2:
            print(f'error with {line}')
            continue
        undiagnosed_emojis.append(int(temp[1]))
    for line in demojifile:
        temp = line.split(':')
        if len(temp) != 2:
            print(f'error with {line}')
            continue
        
        diagnosed_emojis.append(int(temp[1]))

    unemojifile.close()
    demojifile.close()
    return diagnosed_emojis, undiagnosed_emojis

In [ ]:
import matplotlib.pyplot as plt
demoji_list, unmoji_list = emoji_distributions()
plt.hist(demoji_list, range = (0, 600))
#plt.hist(unmoji_list)
plt.show()

In [ ]:
#temp cell to find users who i have data on and print that list to a file / store it in a list

dpath = 'demoji_diagnosed/'
upath = 'demoji_undiagnosed/'

unlist = os.listdir(unpath)
dlist = os.listdir(dpath)

print(len(unlist))
print(len(dlist))

duser_list = []
uuser_list = []
ufilename = 'current_undiagnosed_users.csv'
ufile = open(ufilename, 'w')
for user in unlist:
    if '_submission' in user:
        name = user.split('_submission')[0]
    elif '_comment' in user:
        name = user.split('_comment')[0]
    else:
        print(f'error on {user}')
        continue
    if name in uuser_list:
        continue
    else:
        uuser_list.append(name)
        ufile.write(str(name))
        ufile.write(':;')
ufile.close()

dfilename = 'current_diagnosed_users.csv'
dfile = open(dfilename, 'w')
for user in dlist:
    if '_submission' in user:
        name = user.split('_submission')[0]
    elif '_comment' in user:
        name = user.split('_comment')[0]
    else:
        print(f'error on {user}')
        continue
    if name in duser_list:
        continue
    else:
        duser_list.append(name)
        dfile.write(str(name))
        dfile.write(':;')
dfile.close()

print(f'number of diagnosed users currently: {len(duser_list)}')
print(f'number of undiagnosed users currently {len(uuser_list)}')

In [ ]:
#3738 undiagnosed files
#15672 diagnosed files
#9954 diagnosed users
#1870 undiagnosed users
unpath = 'undiagnosed_user_data/'
dpath = 'diagnosed_user_data/'
dpath2 = 'diagnosed_current/'
unpath2 = 'undiagnosed_current/'
dpath3 = 'demoji_diagnosed/'
unpath3 = 'demoji_undiagnosed/'

unlist1 = os.listdir(unpath)
dlist1 = os.listdir(dpath)
unlist2 = os.listdir(unpath2)
dlist2 = os.listdir(dpath2)
unlist3 = os.listdir(unpath3)
dlist3 = os.listdir(dpath3)

print(f'first undiagnosed count: {len(unlist1)}')
print(f'first diagnosed count: {len(dlist1)}')
print(f'second undiagnosed count: {len(unlist2)}')
print(f'second diagnosed count: {len(dlist2)}')
print(f'third undiagnosed count: {len(unlist3)}')
print(f'third diagnosed count: {len(dlist3)}')

print(len(dsubmissions))
print(len(dcomments))
print(f'combined diagnosed: {len(dsubmissions) + len(dcomments)}')
print(len(usubmissions))
print(len(ucomments))
print(f'combined undiagnosed: {len(usubmissions) + len(ucomments)}')

In [ ]:
from spellchecker import SpellChecker
from nltk.stem import PorterStemmer
import string

def remove_whitespaces(line):
    return " ".join(line.split())

def spell_correction(text):
    wordlist = text.split()
    corrected_wordlist = []
    checker = SpellCheck()
    for word in wordlist:
        correct = checker.correction(word)
        corrected_wordlist.append(correct)
    return " ".join(corrected_wordlist)

def stemming(text):
    wordlist = text.split()
    corrected_wordlist = []
    porter = PorterStemmer()
    for word in wordlist:
        correct = porter.stem(word)
        corrected_wordlist.append(correct)
    return " ".join(corrected_wordlist)
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_punct(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def preprocessv1(text):
    text = remove_urls(text)
    text = remove_punct(text)
    text = remove_whitespaces(text) #removing extra white spaces
    text = text.lower() #lowercasing
    
    return text
    
    

In [ ]:
#pandas part 1, make a dataframe with username: combined text

dpath3 = 'demoji_diagnosed/'
unpath3 = 'demoji_undiagnosed/'
drawframe = pd.DataFrame()
drawdict = {}
urawframe = pd.DataFrame()
urawdict = {}
unlist3 = os.listdir(unpath3)
dlist3 = os.listdir(dpath3)

for username in unlist3:
    filename = 'demoji_undiagnosed/' + username
    demojifile = open(filename, 'r')
    if '_submission' in username:
        for line in demojifile:
            linelist = line.split(':;')
            if len(linelist) != 15:
                continue
            author = linelist[0]
            selftext = linelist[9]
            selftext += ' '
            selftext += linelist[14]
            selftext = preprocessv1(selftext)
            if author in urawdict:
                urawdict[author] += ' ' + selftext
                urawframe[author] += ' ' + selftext
            else:
                urawdict[author] = selftext
                urawframe[author] = selftext
                
    elif '_comment' in username:
        for line in demojifile:
            linelist = line.split(':;')
            if len(linelist) != 12:
                continue
            author = linelist[0]
            body = linelist[2]
            body = preprocessv1(body)
            if author in urawdict:
                urawdict[author] += ' ' + body
                urawframe[author] += ' ' + body
            else:
                urawdict[author] = body
                urawframe[author] = body
    
for username in dlist3:
    filename = 'demoji_diagnosed/' + username
    demojifile = open(filename, 'r')
    
    if '_submission' in username:
        for line in demojifile:
            linelist = line.split(':;')
            if len(linelist) != 15:
                continue
            author = linelist[0]
            selftext = linelist[9]
            selftext += ' '
            selftext += linelist[14]
            selftext = preprocessv1(selftext)
            if author in drawdict:
                drawdict[author] += ' ' + selftext
                drawframe[author] += ' ' + selftext 
            else:
                drawdict[author] = selftext
                drawframe[author] = selftext
    elif '_comment' in username:
        for line in demojifile:
            linelist = line.split(':;')
            if len(linelist) != 12:
                continue
            author = linelist[0]
            body = linelist[2]
            body = preprocessv1(body)
            if author in drawdict:
                drawdict[author] += ' ' + body
                drawfame[author] += ' ' + body
            else:
                drawdict[author] = body
                drawframe[author] = body
    
                
ddictframe = pd.DataFrame(drawdict)
udictframe = pd.DataFrame(urawdict)

In [ ]:
#diagnosed part only
dpath3 = 'demoji_diagnosed/'
drawframe = pd.DataFrame()
drawdict = {}
dlist3 = os.listdir(dpath3)


for username in dlist3:
    filename = 'demoji_diagnosed/' + username
    demojifile = open(filename, 'r')
    
    if '_submission' in username:
        for line in demojifile:
            linelist = line.split(':;')
            if len(linelist) != 15:
               
                continue
            author = linelist[0]
            selftext = linelist[9]
            selftext += ' '
            selftext += linelist[14]
            selftext = preprocessv1(selftext)
            if author in drawdict:
                drawdict[author] += ' ' + selftext
               # drawframe[author] += ' ' + selftext 
            else:
                drawdict[author] = selftext
                #drawframe[author] = selftext
    elif '_comment' in username:
        for line in demojifile:
            linelist = line.split(':;')
            if len(linelist) != 12:
               
                continue
            author = linelist[0]
            body = linelist[2]
            body = preprocessv1(body)
            if author in drawdict:
                drawdict[author] += ' ' + body
                #drawframe[author] += ' ' + body
            else:
                drawdict[author] = body
               # drawframe[author] = body

                
#ddictframe = pd.DataFrame(drawdict)
print('done')


In [ ]:
ddictframe = pd.DataFrame(drawdict, index = [0])
ddictframe = ddictframe.transpose()
print(ddictframe.head())
ddictframe.to_csv('ddframe.csv', sep = ';')

In [ ]:
print(len(dlist3))
print(len(ddictframe))

In [ ]:
#diagnosed part only retrieves subreddits
dpath3 = 'demoji_diagnosed/'
dlist3 = os.listdir(dpath3)

dusers_persub = {}
dposts_persub = {}
dwordcounts_persub = {}
processed_subs = [] #stores a list of initialized subs so i dont have to check each dict

for username in dlist3:
    filename = 'demoji_diagnosed/' + username
    demojifile = open(filename, 'r')
    
    if '_submission' in username:
        for line in demojifile:
            linelist = line.split(':;')
            if len(linelist) != 15:
               
                continue
            user = linelist[0]
            selftext = linelist[9]
            selftext += ' '
            selftext += linelist[14]
            selftext = preprocessv1(selftext)
            subreddit = linelist[10]
            wordcount = len(selftext.split(' '))
            
            
            if subreddit in processed_subs:
                dposts_persub[subreddit] += 1
                dwordcounts_persub[subreddit] += wordcount
                
                if user in dusers_persub[subreddit]:
                    dusers_persub[subreddit][user] += 1
                else:
                    dusers_persub[subreddit][user] = 1
            else:
                dposts_persub[subreddit] = 1
                dwordcounts_persub[subreddit] = wordcount
                dusers_persub[subreddit] = {}
                dusers_persub[subreddit][user] = 1
                processed_subs.append(subreddit)
           
    elif '_comment' in username:
        for line in demojifile:
            linelist = line.split(':;')
            if len(linelist) != 12:
               
                continue
            user = linelist[0]
            body = linelist[2]
            body = preprocessv1(body)
            subreddit = linelist[9]
            wordcount = len(body.split(' '))
           
            if subreddit in processed_subs:
                dposts_persub[subreddit] += 1
                dwordcounts_persub[subreddit] += wordcount
                
                if user in dusers_persub[subreddit]:
                    dusers_persub[subreddit][user] += 1
                else:
                    dusers_persub[subreddit][user] = 1
            else:
                dposts_persub[subreddit] = 1
                dwordcounts_persub[subreddit] = wordcount
                dusers_persub[subreddit] = {}
                dusers_persub[subreddit][user] = 1
                processed_subs.append(subreddit)
           
print('done')


In [ ]:
#diagnosed part only retrieves subreddits with users
dpath3 = 'demoji_diagnosed/'
dlist3 = os.listdir(dpath3)

dusers_persub = {}
processed_subs = [] #stores a list of initialized subs so i dont have to check each dict
processed_users = {} #one list per sub

for username in dlist3:
    filename = 'demoji_diagnosed/' + username
    demojifile = open(filename, 'r')
    
    if '_submission' in username:
        for line in demojifile:
            linelist = line.split(':;')
            if len(linelist) != 15:
                continue
            user = linelist[0]
            subreddit = linelist[10]
            
            if subreddit in processed_subs:
                if not user in processed_users[subreddit]:
                
                    dusers_persub[subreddit] += 1
                    processed_users[subreddit].append(user)
                
            else:
                processed_subs.append(subreddit)
                dusers_persub[subreddit] = 1
                processed_users[subreddit] = []
                processed_users[subreddit].append(user)
                
           
    elif '_comment' in username:
        for line in demojifile:
            linelist = line.split(':;')
            if len(linelist) != 12:
               
                continue
            user = linelist[0]
           
            subreddit = linelist[9]
          
           
            if subreddit in processed_subs:
                if not user in processed_users[subreddit]:
                    dusers_persub[subreddit] += 1
                    processed_users[subreddit].append(user)
                
            else:
                processed_subs.append(subreddit)
                dusers_persub[subreddit] = 1
                processed_users[subreddit] = []
                processed_users[subreddit].append(user)
           
print('done')


In [ ]:
dpostframe = pd.DataFrame(dposts_persub, index = [0])
dpostframe = dpostframe.transpose()
dpostframe.to_csv('dpostframe.csv', sep = ';')
print(dpostframe.head())

In [ ]:
dwordframe = pd.DataFrame(dwordcounts_persub, index = [0])
dwordframe = dwordframe.transpose()
dwordframe.to_csv('dwordframe.csv', sep = ';')
print(dwordframe.head())

In [ ]:
print(len(dwordcount_persub))
print(len(dposts_persub))

In [ ]:
duserframe = pd.DataFrame(dusers_persub, index = [0])
#duserframe = duserframe.transpose()
duserframe.to_csv('duserframe.csv', sep = ';')
print(duserframe.head())

In [ ]:
from operator import itemgetter
res = dict(sorted(dusers_persub.items(), key = itemgetter(1), reverse = True)[100:200])
tempfile = open('dusers100to200.csv', 'w')
for name in res:from operator import itemgetter
res = dict(sorted(dusers_persub.items(), key = itemgetter(1), reverse = True)[100:200])
tempfile = open('dusers100to200.csv', 'w')
for name in res:
    tempfile.write(str(name))
    tempfile.write(';')
    tempfile.write(str(res[name]))
    tempfile.write('\n')
tempfile.close()
    tempfile.write(str(name))
    tempfile.write(';')
    tempfile.write(str(res[name]))
    tempfile.write('\n')
tempfile.close()

In [ ]:
def csv_to_text(filename):
    csvfilename = '' + filename + '.csv'
    clonefilename = '' + filename + '.txt'
    csvfile = open(csvfilename, 'r')
    clonefile = open(clonefilename, 'w')
    for line in csvfile:
        clonefile.write(line.strip())
        clonefile.write(';')
        clonefile.write('\n')
    csvfile.close()
    clonefile.close()

In [ ]:
#csv_to_text('dposts')
csv_to_text('dusers100to200')

In [ ]:
def get_stemmed_data(extension):
    dfilename = 'dstemmed_' + extension + '.txt'
    ufilename = 'ustemmed_' + extension + '.txt'
    
    ddict = {}
    udict = {}
    
    dfile = open(dfilename,'r')
    ufile = open(ufilename,'r')
    
    for line in dfile:
        line_list = line.split(':;')
        if len(line_list) != 2:
            print('error')
            break
        ddict[line_list[0]] = line_list[1].strip() 
    
    for line in ufile:
        line_list = line.split(':;')
        if len(line_list) != 2:
            print('error')
            break
        udict[line_list[0]] = line_list[1].strip() 
    
    dfile.close()
    ufile.close()
    
    return ddict, udict

In [ ]:
import numpy as np

def create_fasttext_file(extension):
    ddict, udict = get_stemmed_data(extension)
    newfilename = 'fasttext_' + extension + '.txt'
    newfile = open(newfilename, 'w')
    for user in ddict:
        label = '__label__diagnosed'
        newfile.write(str(ddict[user]).strip())
        newfile.write(' ')
        newfile.write(label)
        newfile.write('\n')
        
    for user in udict:
        label = '__label__undiagnosed'
        newfile.write(str(ddict[user]).strip())
        newfile.write(' ')
        newfile.write(label)
        newfile.write('\n')
    
    newfilename.close()
def split_fasttext_file(extension, test_decimal):
    basefilename = 'fasttext_' + extension + '.txt'
    basefile = open(basefilename, 'r')
    baselist = list(basefile.readlines())
    basefile.close()
    np.random.shuffle(baselist)
    trainfilename = 'fasttext_' + extension + 'train.txt'
    testfilename = 'fasttext_' + extension + 'test.txt'
    
    split_index = (1.0-test_decimal) * len(baselist)
    train_split = baselist[::split_index-1]
    test_split = baselist[split_index::]
    
    trainfile = open(trainfilename, 'w')
    for line in train_split:
        trainfile.write(str(train_split[line]).strip())
        trainfile.write('\n')
    trainfile.close()
    
    testfile = open(testfilename, 'w')
    for line in test_split:
        testfile.write(str(test_split[line]).strip())
        testfile.write('\n')
    testfile.close()